<a href="https://colab.research.google.com/github/Bindu3sriya/bitcoin_pridction/blob/main/bitcoin_price_prediction_and_notifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import datetime
import requests
import smtplib
from email.mime.text import MIMEText

In [37]:
data = pd.read_csv('https://raw.githubusercontent.com/Bindu3sriya/dataset/main/BTC-USD.csv')


In [38]:
data.dropna(subset=['Date', 'Close'], inplace=True)


In [39]:
data['Date'] = pd.to_datetime(data['Date'])
data['DayOfWeek'] = data['Date'].dt.dayofweek

In [40]:
train_data, val_data = train_test_split(data, test_size=0.2, shuffle=False)

In [41]:
X_train = train_data[['DayOfWeek']]  # Features
y_train = train_data['Close']         # Target

In [42]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [43]:
X_val = val_data[['DayOfWeek']]
y_val = val_data['Close']

In [44]:
y_val_pred = model.predict(X_val)
val_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
print(f'Validation RMSE: {val_rmse}')

Validation RMSE: 6997.926870605082


In [45]:
user_threshold = float(input("Enter your desired price threshold: "))

Enter your desired price threshold: 855


In [46]:
next_day = [[6]]  # Wrap the feature value in a list of lists
predicted_price = model.predict(next_day)[0]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [47]:
# Check if predicted price is below user threshold
if predicted_price <= user_threshold:
    # Calculate the date for the next Sunday
    today = datetime.date.today()
    days_until_sunday = (6 - today.weekday()) % 7
    next_sunday = today + datetime.timedelta(days=days_until_sunday)

    # Send notification to user
    notification_message = (
        f"Predicted Bitcoin price is below the threshold on {next_sunday}. "
        f"Predicted price: {predicted_price:.2f} USD."
    )

    # Send WhatsApp notification
    def send_whatsapp_notification(message):
        api_url = 'https://api.callmebot.com/whatsapp.php?phone=+919573639109&apikey=2278832&text=' + message
        response = requests.post(api_url)

        if response.status_code == 200:
            print("WhatsApp notification sent successfully.")
        else:
            print("Error sending WhatsApp notification.")

    send_whatsapp_notification(notification_message)
    #send_email_notification('Bitcoin Price Alert', notification_message)

print("Prediction and notification complete.")

Prediction and notification complete.
